# 🛠️ Add Your Own Algorithm to IGL-Bench

The following shows the steps required to register a brand‑new algorithm inside IGL_Bench.

> Everything else (data loading, masking, seeds, config loading) is already handled by the framework.

## 📁 Step 1: Folder layout and file names

IGL_Bench discovers solvers and configs **by name**(string). Create the following two files:

```
IGL_Bench/
  algorithm/
    MyGNN/
      __init__.py
      solver.py         # ← implements MyGNN_node_solver or MyGNN_graph_solver
config/
  topo_global/
    MyGNN.yml           # ← hyper-parameter config
```

> Replace topo_global with class, topo_local, or topology to match your imbalance type.

## 📄 Step 2: Write config/topo_global/MyGNN.yml

In [ ]:
yaml_content = '''\
algorithm: "MyGNN"
task: "node"

lr: 0.005
hidden_dim: 128
dropout: 0.5
weight_decay: 0.0005
epoch: 300
least_epoch: 40

n_layer: 2
'''
print(yaml_content)

## 🧠 Step 3: Implement algorithm/MyGNN/solver.py

The solver must expose a class `<Algorithm>_<task>_solver` and return acc/bacc/mf1/roc from `test()`.

### 📦 Dataset object

In `__init__`, the provided `dataset` contains the following preprocessed attributes:
- `x`, `y`, `adj`, `edge_index`, `adj_norm` (PyTorch tensor format)
- `train_mask`, `val_mask`, `test_mask` (boolean tensors)
- `train_index`, `val_index`, `test_index` (list arrays)

These are ready to use and no additional preprocessing is needed.

In [ ]:
import torch
import torch.nn.functional as F
from IGL_Bench.backbone.gcn import GCN_node_sparse
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score

class MyGNN_node_solver:
    def __init__(self, config, dataset, device="cuda"):
        self.cfg = config
        self.data = dataset.to(device)
        self.device = device
        n_feat = self.data.num_features
        n_class = int(self.data.y.max().item() + 1)
        self.model = GCN_node_sparse(n_feat, self.cfg.hidden_dim, n_class,
                                     self.cfg.n_layer, self.cfg.dropout).to(device)
        self.opt = torch.optim.Adam(self.model.parameters(), lr=self.cfg.lr,
                                    weight_decay=self.cfg.weight_decay)

    def reset_parameters(self):
        if hasattr(self.model, "reset_parameters"):
            self.model.reset_parameters()
        for g in self.opt.param_groups:
            g["lr"] = self.cfg.lr

    def train(self):
        self.reset_parameters()
        patience, counter, best = 10, 0, 0.0
        for epoch in range(1, self.cfg.epoch + 1):
            self.model.train(); self.opt.zero_grad()
            logits = self.model(self.data.x, self.data.edge_index)
            loss = F.cross_entropy(logits[self.data.train_mask], self.data.y[self.data.train_mask])
            loss.backward(); self.opt.step()
            acc_val = self._eval("val")
            if acc_val > best:
                best, counter = acc_val, 0
            else:
                counter += 1
            if counter >= patience and epoch >= self.cfg.least_epoch:
                print(f"Early stop at epoch {epoch}"); break

    def _eval(self, split="val"):
        self.model.eval()
        mask = getattr(self.data, f"{split}_mask")
        with torch.no_grad():
            logits = self.model(self.data.x, self.data.edge_index)[mask]
        pred = logits.argmax(dim=1).cpu(); true = self.data.y[mask].cpu()
        return accuracy_score(true, pred)

    def test(self):
        self.model.eval(); mask = self.data.test_mask
        with torch.no_grad():
            logits = self.model(self.data.x, self.data.edge_index)[mask]
        pred = logits.argmax(dim=1).cpu()
        prob = F.softmax(logits, dim=1).cpu()
        y = self.data.y[mask].cpu()
        acc  = accuracy_score(y, pred)
        bacc = balanced_accuracy_score(y, pred)
        mf1  = f1_score(y, pred, average="macro")
        roc  = roc_auc_score(y, prob, multi_class="ovr", average="macro")
        return acc, bacc, mf1, roc

## 🚀 Step 4: Smoke test your algorithm
After saving solver.py and MyGNN.yml, test them with the Manager.

In [ ]:
import IGL_Bench as igl

dataset = igl.dataset.Dataset(
    task="node",
    data_name="Cora",
    imb_type="topo_global",
    imb_level="high"
).load_dataset()

cfg = igl.config.load_conf(task="node", imbtype="topo_global", algorithm="MyGNN")
solver = igl.manage.Manager(cfg, dataset)
solver.run(num_runs=3)

## ✅ Summary
- Your solver.py needs to define __init__(), train(), reset_parameters(), and test()
- Configs are loaded dynamically using the algorithm name
- Return (acc, bacc, macro_f1, auc_roc) in order

Your algorithm is now benchmark-ready!